<a href="https://colab.research.google.com/github/SebastienBienfait/L2C-Data-managment/blob/main/Data_jobs_reed(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This finds all the job codes of a search of Data - Ashford, Kent - radius=30 on Reed.co.uk

and for the search: Data - Kent - radius = 20

In [ ]:
x = np.array([1,2,3])
a = np.array([3,2,1])

x = np.append(x,[a])
print(x)

In [2]:
### My reed api_key: 96a7ec49-549c-4529-b9d2-fa3059a437b3
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


#pages = np.range(63)

#url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno=63&proximity=30"
url_ashford_30 = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?proximity=30"
url_kent_20 = "https://www.reed.co.uk/jobs/data-jobs-in-kent?proximity=20"
initial_url = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?proximity=30"
def get_html(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup


def find_jobs_iterate(url):
  site_html = get_html(url)
  results = site_html.find(class_="col-sm-8 col-md-9 results-container")

  id_list = np.array([])
  jobs_found = results.find_all(class_="job-result-card")

  for job in jobs_found:
      job_id = int(job["id"].split("jobSection")[1])
      
      id_list = np.append(id_list,[job_id])
  return id_list
  
def find_total_pages(site_html):
  of_total_jobs = site_html.find(class_ = "search-results-footer").text
  #print(of_total_jobs)
  total_jobs_split1 = of_total_jobs.split(" jobs")[0]
  total_jobs = total_jobs_split1.split("of ")[1]
  print(total_jobs)
#not used

#soup = get_html(url)
#id_list = find_jobs(soup)

#print(len(id_list))

In [ ]:
from pandas.core.arrays.sparse import dtype

def the_scrape(url):
  soup = get_html(url)
  results = soup.find(class_="col-sm-8 col-md-9 results-container")

  total_id_list = np.array([],dtype=int)
  jobs_found = results.find_all(class_="job-result-card")
  #print(jobs_found)
  for job in jobs_found:
      job_id = int(job["id"].split("jobSection")[1])
      
      total_id_list = np.append(total_id_list, [job_id])              
  print(total_id_list)


  of_total_jobs = results.find(class_ = "search-results-footer").text       #
  total_jobs_split1 = of_total_jobs.split(" jobs")[0]                       #Finds the total number of jobs 
  total_jobs_str = total_jobs_split1.split("of ")[1]                        #and total number of pages needed to run
  total_jobs_list = total_jobs_str.split(",")
  total_pages = int(np.ceil(int("".join(map(str,total_jobs_list)) )/25))

  
  print("total pages", total_pages)
  print("1")
  for page_no in np.arange(2,total_pages+1)[:80]:
    print(page_no)
    url2 = "https://www.reed.co.uk/jobs/data-jobs-in-ashford-kent?pageno="+str(page_no)+"&proximity=30"
    print(url2)
    page_id_list = find_jobs_iterate(url2)

    total_id_list = np.append(total_id_list,[page_id_list])
  return total_id_list

#id_list = the_scrape(url_ashford_30)
id_list = the_scrape(url_kent_20)
id_list = id_list.astype("int")
#print(id_list)



In [4]:
print(len(id_list))

5241


In [5]:
def call_api(id_list):
  base_url = "https://www.reed.co.uk/api/1.0/jobs/"
  url = ""
  job_df = pd.DataFrame()


  for job_id in id_list:
    url = base_url+str(job_id)
    post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3",""))
    json_data = post_request.json()


    new_row = pd.json_normalize(json_data)
    job_df = job_df.append(new_row,ignore_index=True)
  return job_df
full_job_df = call_api(id_list)
display(full_job_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5241 entries, 0 to 5240
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   employerId           2000 non-null   float64
 1   employerName         2000 non-null   object 
 2   jobId                2000 non-null   float64
 3   jobTitle             2000 non-null   object 
 4   locationName         2000 non-null   object 
 5   minimumSalary        1581 non-null   object 
 6   maximumSalary        1581 non-null   object 
 7   yearlyMinimumSalary  1581 non-null   object 
 8   yearlyMaximumSalary  1581 non-null   object 
 9   currency             1581 non-null   object 
 10  salaryType           2000 non-null   object 
 11  salary               1581 non-null   object 
 12  datePosted           2000 non-null   object 
 13  expirationDate       2000 non-null   object 
 14  externalUrl          343 non-null    object 
 15  jobUrl               2000 non-null   o

None

In [18]:
print(full_job_df.columns)

#job_df = full_job_df.drop(columns = ['employerId','minimumSalary', 'maximumSalary', 'yearlyMinimumSalary',
#       'yearlyMaximumSalary', 'currency', 'salaryType', 'datePosted',
#       'expirationDate','partTime', 'fullTime', 'jobDescription', 'applicationCount',"message","jobId"])

job_df = full_job_df[["jobTitle", "employerName", "salary", "contractType", "locationName",  "jobUrl"]]


drop_list = np.arange(2000,5241)

job_df.drop(index=drop_list,inplace = True)
job_df.drop_duplicates(inplace=True)

display(job_df)

Index(['employerId', 'employerName', 'jobId', 'jobTitle', 'locationName',
       'minimumSalary', 'maximumSalary', 'yearlyMinimumSalary',
       'yearlyMaximumSalary', 'currency', 'salaryType', 'salary', 'datePosted',
       'expirationDate', 'externalUrl', 'jobUrl', 'partTime', 'fullTime',
       'contractType', 'jobDescription', 'applicationCount', 'message'],
      dtype='object')


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,jobTitle,employerName,salary,contractType,locationName,jobUrl
0,Data Architect,The Rank Group,None,Permanent,London,https://www.reed.co.uk/jobs/data-architect/484...
1,Data Engineer,Veolia UK,"£40,000 - £56,000 per annum, inc benefits",Permanent,London,https://www.reed.co.uk/jobs/data-engineer/4853...
2,Data Analyst,Commercial Services Interim & Executive Search,"£41,216 - £47,199 per annum",Permanent,West Malling,https://www.reed.co.uk/jobs/data-analyst/48529572
3,Data Lead,Reed,"£27,000 - £39,000 per annum, inc benefits",Permanent,Folkestone,https://www.reed.co.uk/jobs/data-lead/48401295
4,Data Analyst,Morgan Sindall Property Services,"£35,000 - £40,000 per annum, inc benefits",Permanent,Dartford,https://www.reed.co.uk/jobs/data-analyst/48211407
...,...,...,...,...,...,...
1993,Research Manager,Aspire,"£35,000 - £45,000 per annum",Permanent,London,https://www.reed.co.uk/jobs/research-manager/4...
1994,Business Development Assistant,STRIDE RESOURCE MANAGEMENT LTD,"£25,000 - £30,000 per annum",Permanent,London,https://www.reed.co.uk/jobs/business-developme...
1996,Paid Search Marketing Executive £25K-£30K Me...,GradBay,"£25,000 - £30,000 per annum",Permanent,London,https://www.reed.co.uk/jobs/paid-search-market...
1997,Technical Lead - DB Pensions,WTW,None,Permanent,Redhill,https://www.reed.co.uk/jobs/technical-lead-db-...


NB: very few jobs have 'Data' in the title past index 65. Only 3 further jobs appear with relevent information, 154, 209, 1553. I find it unlikly that an exorstive search of rows 2000+ will give many more relevent jobs.

In [19]:
job_df.to_csv("reed_data_jobs_expandedSearch.csv")

In [ ]:
#@title Default title text
###Failed
"""
key = "96a7ec49-549c-4529-b9d2-fa3059a437b3"

exm_url = """https://www.reed.co.uk/api/{versionnumber}/search?keywords={keywords}&loc 
ationName={locationName}&employerId={employerId}&distanceFromLocation={distance in miles}"""


url = "https://www.reed.co.uk/api/1.0/search?keywords=data&location=ashfordkent&distanceFromLocation=10" #

post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3","")) #.json()#['totalResults']

if post_request.status_code == 200:
  print("connection made, code: ", post_request.status_code)
  json_data = post_request.json()
else:
  print("connection fail, error code: ", post_request.status_code)

json_keys = json_data.keys()
print("total results found: ",json_data["totalResults"])
#'results' has the top 100 results from the api
#'ambiguousLocations' is empty
#'totalResults' is: 43984, the total value found by the api
print(json_keys) #.keys()
2
results_df = pd.json_normalize(json_data["results"]) 
#ambi_df = pd.json_normalize(json_data["ambiguousLocations"])
display(results_df.info())

results_df.drop(columns = ["minimumSalary", 'currency', 'expirationDate', 'date','jobDescription',
       'applications',"employerProfileName","employerProfileId","maximumSalary"],inplace=True)


def outside_kent():
  drop_cities = ["Bristol", "Manchester","Colchester","Reading","Brussels","Glasgow","Edinburgh","Southampton","Oxford","Cambridge"]
  for city in drop_cities:
    results_df.drop(results_df[results_df["locationName"] == city].index, inplace =True)
#outside_kent()

#results_df.drop("locationName")
display(results_df)
less_results_df = results_df[['employerName', 'jobTitle' ,'locationName']]
#display(less_results_df)
"""

In [ ]:
###api test

job_id = "48445740"
print(job_id)
url = "https://www.reed.co.uk/api/1.0/jobs/" # 48401296

url = url+job_id
post_request = requests.get(url, auth=("96a7ec49-549c-4529-b9d2-fa3059a437b3","")) #.json()#['totalResults']
if post_request.status_code == 200:
  print("connection made, code: ", post_request.status_code)
  json_data = post_request.json()
else:
  print("connection fail, error code: ", post_request.status_code)
print(json_data)

df = pd.json_normalize(json_data)
df


